In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [4]:
import sys
sys.path.append('..')
import importlib  
import pandas as pd
from andrew.util.prescribe_helper import prescribe_help
val = importlib.import_module("covid-xprize-robotasks-main.judging.scripts.prescribe.prescriptor_validation")


In [ ]:
import tensorflow as tf

In [ ]:
class TestLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.w = tf.Variable(
            initial_value = tf.constant(2.0, dtype='float32'),
            trainable=True)
    def call(self, inputs):
        return tf.math.multiply(inputs, self.w)
            

In [ ]:
import time

In [ ]:
a = tf.keras.layers.Input(shape=(), name="test")
test_layers = [TestLayer()(a)]
test_layers_2 = [TestLayer()(a)]
for i in range(100):
    test_layers.append(TestLayer()(test_layers_2[-1]))
    test_layers_2.append(TestLayer()(test_layers[-1]))
    
#b = TestLayer()(a)
mod = tf.keras.models.Model(inputs=a, outputs=[test_layers[-1], test_layers_2[-1]])
inp = tf.constant([10.0])
opt = tf.keras.optimizers.SGD()

In [ ]:
@tf.function
def non_exp():
    with tf.GradientTape() as tape:
        z = mod(inp)
        loss = z
        gradients = tape.gradient(loss, mod.trainable_weights)
        opt.apply_gradients(zip(gradients, mod.trainable_weights))

In [ ]:
start = time.time()
non_exp()
end = time.time()
print(end-start)

In [ ]:
start = time.time()
for i in range(10000):
    non_exp()
end = time.time()
print(end-start)

In [ ]:
a_x = tf.keras.layers.Input(shape=(), name="test_x")
test_layers_x = [TestLayer()(a_x)]
test_layers_2_x = [TestLayer()(a_x)]
for i in range(100):
    test_layers_x.append(TestLayer()(test_layers_2_x[-1]))
    test_layers_2_x.append(TestLayer()(test_layers_x[-1]))
    
#b = TestLayer()(a)
mod_x = tf.keras.models.Model(inputs=a_x, outputs=[test_layers_x[-1], test_layers_2_x[-1]])
inp_x = tf.constant([10.0])
opt_x = tf.keras.optimizers.SGD()

In [ ]:
@tf.function(experimental_compile=True)
def exp():
    with tf.GradientTape() as tape:
        z = mod_x(inp_x)
        loss = z
        gradients = tape.gradient(loss, mod_x.trainable_weights)
        opt_x.apply_gradients(zip(gradients, mod_x.trainable_weights))

In [ ]:
start = time.time()
exp()
end = time.time()
print(end-start)

In [ ]:
start = time.time()
for i in range(10000):
    exp()
end = time.time()
print(end-start)

In [ ]:
! python test.py

In [ ]:
prescribe_help("2021-02-01", "2021-02-03", '../covid-xprize-robotasks-main/ips/prescriptions/uk_future_test_ips.csv', '../covid-xprize-robotasks-main/ip_costs/uniform_random_costs.csv', 'test.csv')

In [12]:
tasks = pd.read_csv('../covid-xprize-robotasks-main/tasks/prescribe_tasks.csv')

In [14]:
tasks

,StartDate,EndDate,IpFile,CostFile,OutputFile
0,2021-02-15,2021-05-15,~/covid-xprize-robotasks-main/ips/prescription...,~/covid-xprize-robotasks-main/ip_costs/fixed_e...,~/work/prescriptions/uk_future_test_task.csv
1,2021-01-01,2021-03-31,~/covid-xprize-robotasks-main/ips/prescription...,~/covid-xprize-robotasks-main/ip_costs/uniform...,~/work/prescriptions/all_2021q1_test_task.csv


In [15]:
task = tasks.iloc[0]
start_date = "2021-02-01"#task["StartDate"]
end_date = "2021-02-03"#task["EndDate"]
ip_file = task["IpFile"]
#submission_file = 'prescriptions/all_2021q1_test_task.csv'
submission_file = 'andrew/test.csv'

In [23]:
val.validate_submission(start_date, end_date, ip_file, submission_file)

In [24]:
! python ../covid-xprize-robotasks-main/judging/scripts/prescribe/prescriptor_validation.py -s {start_date} -e {end_date} -ip {ip_file} -f {submission_file}